[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Fine-Tune YOLO for Emergency Vehicle Detection**

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## 🎯 Objective

This notebook provides a step-by-step guide to fine-tuning a YOLO segmentation model to detect emergency vehicles, specifically ambulances. The goal is to create a model that can accurately identify and segment these vehicles in images and videos, which is a crucial component for intelligent traffic management systems.


## 🚀 Key Features

* **Dataset Sourcing**: Guidance on where to find and collect images of emergency vehicles.
* **Data Annotation**: Steps for annotating the dataset with segmentation masks using an external tool.
* **Format Conversion**: Conversion of standard COCO JSON annotations into the YOLO segmentation format.
* **Model Training**: Fine-tuning a pre-trained YOLOv11 segmentation model on the custom ambulance dataset.
* **Video Inference**: Applying the trained model to a video to detect and segment emergency vehicles in real-time.


## 📚 Libraries & Prerequisites

* **Core Libraries**: `ultralytics`.
* **Environment**: A Python environment with GPU support (like Google Colab) is highly recommended for efficient model training.
* **Dataset**: A custom-collected dataset of emergency vehicle images, an annotation file in COCO format, and test videos for inference.


### **Create a Dataset**

The first step is to gather a dataset of images. For this project, we need images of emergency vehicles. The notebook suggests using a specific Kaggle dataset for ambulance images, which serves as our training data.

In [ ]:
# Here you have to collect various images of the object you want to detect

# In our case, we will collect images of a Ambulance

# You can go kaggle to download this dataset of ambulances
link = 'https://www.kaggle.com/datasets/aliwannous2021/ambulance-vehicles?select=ambulance2+%28101%29.jpg'

### **Performed Annotation on the Dataset**

Once the images are collected, they need to be annotated. This involves drawing precise segmentation masks (polygons) around the ambulances in each image. The notebook outlines the process of using an external tool like Labellerr for this task and then exporting the annotations in COCO JSON format.

In [ ]:
# Go to labellerr.com and create a workspace

# Upload the images from the output directory

# Annotate the images with bounding boxes for PPE items

# Export the annotations in COCO JSON format

### **Convert COCO JSON Annotation to YOLO format**

The YOLO model requires annotations in a specific text format, not COCO JSON. We clone a utility repository that contains a helper script (`seg_converter`) to convert our COCO segmentation annotations into the required YOLO format, making them ready for training.

In [ ]:
# Clone the utility repository to access the required functions
!git clone https://github.com/yashsuman15/yolo_finetune_utils.git

In [ ]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

result = coco_to_yolo_converter(
            json_path='./annotation.json',
            images_dir='./dataset',
            output_dir='yolo_format'
            )

### **Install the Ultralytics Package**

We need the `ultralytics` library, which provides the powerful YOLO framework for training and inference. We install it directly using pip.

In [ ]:
!pip install ultralytics

### **Import All the Requried Libraries**

Now, we import the necessary libraries and run a check to ensure `ultralytics` is set up correctly. We then define the file path to our newly formatted dataset directory, which will be passed to the model for training.

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from IPython.display import Image

In [ ]:
!pwd

In [ ]:
location = !pwd
dataset_path = f"{location[0]}/yolo_format"
print(f"Dataset path: {dataset_path}")

### **Train YOLO11 Model on a Custom Dataset**

This is the core step where we train our custom model. We use the `yolo` command-line interface to start a segmentation training task (`task=segment`). We point it to our dataset's `data.yaml` file, use a pre-trained `yolo11x-seg.pt` model as a starting point, and fine-tune it for 200 epochs.

In [ ]:
!yolo task=segment mode=train data={dataset_path}/data.yaml model="yolo11x-seg.pt" epochs=200 imgsz=640 batch=10

### **Inference with Fine-tune on Videos**

After training is complete, we use our best-trained model weights (`best.pt`) to run inference on a test video. We run a prediction task, set a confidence threshold of 0.7, and specify the source video. The `save=True` argument will save the output video with the detected segmentation masks overlaid.

In [ ]:
!yolo task=segment mode=predict model="./runs/segment/train/weights/best.pt" conf=0.7 source="./video/g.mp4" save=True show_labels=False